In [ ]:
!pip install PyDrive


In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import io
import datetime
import numpy as np
from matplotlib import pyplot
import seaborn as sns


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
download = drive.CreateFile({'id': 'xxxxxx'})

download.GetContentFile('new-thyroid.data')

In [ ]:
df = pd.read_csv('new-thyroid.data', low_memory = False, header=None)

In [ ]:
df

,0,1,2,3,4,5
0,1,107,10.1,2.2,0.9,2.7
1,1,113,9.9,3.1,2.0,5.9
2,1,127,12.9,2.4,1.4,0.6
3,1,109,5.3,1.6,1.4,1.5
4,1,105,7.3,1.5,1.5,-0.1
...,...,...,...,...,...,...
210,3,118,6.5,1.3,1.7,11.5
211,3,139,4.2,0.7,4.3,6.3
212,3,103,5.1,1.4,1.2,5.0
213,3,97,4.7,1.1,2.1,12.6


In [ ]:
df.columns = ['output', 'inp1', 'inp2', 'inp3', 'inp4', 'inp5']

In [ ]:
df.output.drop_duplicates()

0      1
150    2
185    3
Name: output, dtype: int64

In [ ]:
len(df[df.output == 3])

30

In [ ]:
df.isnull().sum()

output    0
inp1      0
inp2      0
inp3      0
inp4      0
inp5      0
dtype: int64

In [ ]:
X = df[['inp1', 'inp2', 'inp3', 'inp4', 'inp5']]

In [ ]:
y = df.output

In [ ]:
df2_1 = df[df.output == 1]

In [ ]:
df2_non_1 = df[df.output != 1]

In [ ]:
df2_1_sampled = df2_1.sample(35, random_state=32)

In [ ]:
df_balanced = df2_non_1.append(df2_1_sampled)

In [ ]:
X_last = df_balanced[['inp1', 'inp2', 'inp3', 'inp4', 'inp5']]

In [ ]:
y_last = df_balanced.output

In [ ]:
from sklearn import preprocessing

In [ ]:

from scipy.stats import chi2_contingency

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
fitter = scaler.fit(X_last)
X_train = scaler.transform(X_last)


In [ ]:

from sklearn.neural_network import MLPClassifier

In [ ]:

 clf = MLPClassifier(solver='adam', alpha = 0.1, hidden_layer_sizes=(2), random_state=1, validation_fraction=0, activation='relu')


In [ ]:

 clf2 = MLPClassifier(solver='adam', alpha = 0.1, hidden_layer_sizes=(2), random_state=1, validation_fraction=0.25)

In [ ]:

y_predicted2 = clf2.predict(X_scaled)

In [ ]:

clf2.fit(X_train_scaled,y_last)

In [ ]:

y_predicted_2 = clf2.predict(X_train_scaled)

In [ ]:

from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf2, X_train_scaled, y_last, cv=2)

In [ ]:
scores

array([0.9 , 0.82])

In [ ]:

clf.fit(X_train_scaled,y_last)

In [ ]:

clf = MLPClassifier(solver='adam', alpha = 0.1, hidden_layer_sizes=(3), random_state=1, validation_fraction=0.25)

In [ ]:

y_predicted = clf.predict(X_train_scaled)

In [ ]:

from sklearn.model_selection import train_test_split

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X_train_scaled, y_last, test_size=0.25, random_state=42)

In [ ]:

clf2.fit(X_train, y_train)

In [ ]:

y_pred_cross = clf2.predict(X_test)

In [ ]:

metrics.accuracy_score(y_pred_cross,y_test)

0.92

In [ ]:

metrics.accuracy_score(y_predicted,y_last)

In [ ]:

metrics.accuracy_score(y_predicted_2,y_last)

In [ ]:
y_predicted_proba = clf.predict_proba(X_scaled)
y_predicted = np.argmax(y_predicted_proba, axis=1) + 1
i,j = np.where(y_predicted1 == 1)
j = j + 1
y_predicted = j

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [ ]:

def plot_multiclass_roc(clf, X_test, y_test, y_score, n_classes, figsize=(17, 6)):
    #y_score = clf.decision_function(X_test)

    # structures
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # calculate dummies once
    y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_dummies[:, i], y_score)
        roc_auc[i] = auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic example')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for label %i' % (roc_auc[i], i+1))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    sns.despine()
    plt.show()

In [ ]:

from sklearn.metrics import confusion_matrix

In [ ]:

cnf_matrix = confusion_matrix(y_last, y_predicted)

In [ ]:

cnf_matrix_2 = confusion_matrix(y_last, y_predicted2)

In [ ]:

print(cnf_matrix)

[[23  7  0]
 [ 0 30  5]
 [ 0  1 34]]


In [ ]:

print(cnf_matrix_2)

In [ ]:

plot_multiclass_roc(clf, X_scaled,  y_last, y_predicted_2, 3, figsize=(17, 6))

In [ ]:
!git clone https://github.com/mrthetkhine/RBFNeuralNetwork.git

In [ ]:

metrics.accuracy_score(y_last, y_predicted)

# RBF ile

In [ ]:

import RBFNeuralNetwork

In [ ]:
from keras.models import Sequential
from keras.layers import Dense 

Using TensorFlow backend.


In [ ]:
from keras.layers import Layer
from keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [ ]:
y_output

In [ ]:
from keras import layers

In [ ]:
from keras.layers import Dense, Flatten

In [ ]:
model1 = Sequential()
model1.add(Dense(20, activation='relu', input_dim=5))
#model.add(RBFLayer(10, 0.5))
model1.add(Dense(3, activation='softmax'))


model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
from keras.utils import np_utils

In [ ]:
classes=len(np.unique(y_last))

In [ ]:
Y_train = np_utils.to_categorical(y_last - 1, 3, dtype=int)


In [ ]:
model1.fit(X_train, Y_train,validation_split=0.5,epochs=10)

Train on 50 samples, validate on 50 samples
Epoch 1/10
50/50 [==============================] - 0s 2ms/step - loss: 1.0995 - accuracy: 0.0000e+00 - val_loss: 0.9587 - val_accuracy: 0.7000
Epoch 2/10
50/50 [==============================] - 0s 237us/step - loss: 1.0843 - accuracy: 0.0000e+00 - val_loss: 0.9630 - val_accuracy: 0.7000
Epoch 3/10
50/50 [==============================] - 0s 230us/step - loss: 1.0696 - accuracy: 0.0000e+00 - val_loss: 0.9675 - val_accuracy: 0.7200
Epoch 4/10
50/50 [==============================] - 0s 219us/step - loss: 1.0549 - accuracy: 0.0000e+00 - val_loss: 0.9718 - val_accuracy: 0.7200
Epoch 5/10
50/50 [==============================] - 0s 218us/step - loss: 1.0406 - accuracy: 0.0000e+00 - val_loss: 0.9761 - val_accuracy: 0.7400
Epoch 6/10
50/50 [==============================] - 0s 204us/step - loss: 1.0265 - accuracy: 0.0200 - val_loss: 0.9805 - val_accuracy: 0.7600
Epoch 7/10
50/50 [==============================] - 0s 228us/step - loss: 1.0125 - acc